In [6]:
import pandas as pd
from pyspark.sql import SparkSession
import sys

In [3]:
df = pd.read_csv('project2DataSet.csv')
df.head()

,name,age,exp,salary
0,khader,21.0,1.0,450.0
1,asem,33.0,11.0,600.0
2,moe,55.0,30.0,2000.0
3,yousef,NaN,5.0,3000.0
4,feras,23.0,NaN,300.0


In [9]:
%pip install openpyxl

import pyspark
import pandas as pd
# For Excel files, use read_excel instead of read_csv
pd.read_csv(r"project2DataSet.csv")


Note: you may need to restart the kernel to use updated packages.


,name,age,exp,salary
0,khader,21.0,1.0,450.0
1,asem,33.0,11.0,600.0
2,moe,55.0,30.0,2000.0
3,yousef,NaN,5.0,3000.0
4,feras,23.0,NaN,300.0
5,NaN,36.0,NaN,NaN


In [10]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("project 2") \
    .getOrCreate()


In [11]:
spark

In [13]:
df_spark = spark.read.csv("project2DataSet.csv", header=True, inferSchema=True)

In [15]:
df_spark.show()

+------+----+----+------+
|  name| age| exp|salary|
+------+----+----+------+
|khader|  21|   1|   450|
|  asem|  33|  11|   600|
|   moe|  55|  30|  2000|
|yousef|NULL|   5|  3000|
| feras|  23|NULL|   300|
|  NULL|  36|NULL|  NULL|
+------+----+----+------+



In [23]:
df_spark.drop("name").show()

+----+----+------+
| age| exp|salary|
+----+----+------+
|  21|   1|   450|
|  33|  11|   600|
|  55|  30|  2000|
|NULL|   5|  3000|
|  23|NULL|   300|
|  36|NULL|  NULL|
+----+----+------+



In [24]:
df_spark.show()

+------+----+----+------+
|  name| age| exp|salary|
+------+----+----+------+
|khader|  21|   1|   450|
|  asem|  33|  11|   600|
|   moe|  55|  30|  2000|
|yousef|NULL|   5|  3000|
| feras|  23|NULL|   300|
|  NULL|  36|NULL|  NULL|
+------+----+----+------+



In [ ]:
#all the null values will be dropped
df_spark.na.drop().show()

+------+---+---+------+
|  name|age|exp|salary|
+------+---+---+------+
|khader| 21|  1|   450|
|  asem| 33| 11|   600|
|   moe| 55| 30|  2000|
+------+---+---+------+



In [28]:
### any = how
df_spark.na.drop(how='any').show()

+------+---+---+------+
|  name|age|exp|salary|
+------+---+---+------+
|khader| 21|  1|   450|
|  asem| 33| 11|   600|
|   moe| 55| 30|  2000|
+------+---+---+------+



In [29]:
### any = how
df_spark.na.drop(how='all').show()

+------+----+----+------+
|  name| age| exp|salary|
+------+----+----+------+
|khader|  21|   1|   450|
|  asem|  33|  11|   600|
|   moe|  55|  30|  2000|
|yousef|NULL|   5|  3000|
| feras|  23|NULL|   300|
|  NULL|  36|NULL|  NULL|
+------+----+----+------+



In [ ]:
# threashold (at least 2 non-null values)
df_spark.na.drop(thresh=2).show()

+------+----+----+------+
|  name| age| exp|salary|
+------+----+----+------+
|khader|  21|   1|   450|
|  asem|  33|  11|   600|
|   moe|  55|  30|  2000|
|yousef|NULL|   5|  3000|
| feras|  23|NULL|   300|
+------+----+----+------+



In [31]:
## subset if we wnat to drop null values based on specific columns

df_spark.na.drop(subset=["name", "age"]).show()


+------+---+----+------+
|  name|age| exp|salary|
+------+---+----+------+
|khader| 21|   1|   450|
|  asem| 33|  11|   600|
|   moe| 55|  30|  2000|
| feras| 23|NULL|   300|
+------+---+----+------+



In [42]:
### filling the null values with a specific value
df_spark.na.fill(0,['salary','exp']).show()

+------+----+---+------+
|  name| age|exp|salary|
+------+----+---+------+
|khader|  21|  1|   450|
|  asem|  33| 11|   600|
|   moe|  55| 30|  2000|
|yousef|NULL|  5|  3000|
| feras|  23|  0|   300|
|  NULL|  36|  0|     0|
+------+----+---+------+



In [44]:
from pyspark.sql import functions as F

# Calculate the mean of 'exp' column
exp_mean = df_spark.select(F.mean('exp')).collect()[0][0]
print(f"Mean of 'exp': {exp_mean}")
# Fill null values in 'exp' column with the mean
df_spark.na.fill({'exp': exp_mean}).show()

Mean of 'exp': 11.75
+------+----+---+------+
|  name| age|exp|salary|
+------+----+---+------+
|khader|  21|  1|   450|
|  asem|  33| 11|   600|
|   moe|  55| 30|  2000|
|yousef|NULL|  5|  3000|
| feras|  23| 11|   300|
|  NULL|  36| 11|  NULL|
+------+----+---+------+



In [54]:
from pyspark.ml.feature import Imputer
Imputer = Imputer(
    inputCols=['age', 'salary', 'exp'],
    outputCols=["{}_imputed".format(c) for c in ['age', 'salary', 'exp']],
).setStrategy("mean")

In [55]:
Imputer.fit(df_spark).transform(df_spark).show()

+------+----+----+------+-----------+--------------+-----------+
|  name| age| exp|salary|age_imputed|salary_imputed|exp_imputed|
+------+----+----+------+-----------+--------------+-----------+
|khader|  21|   1|   450|         21|           450|          1|
|  asem|  33|  11|   600|         33|           600|         11|
|   moe|  55|  30|  2000|         55|          2000|         30|
|yousef|NULL|   5|  3000|         33|          3000|          5|
| feras|  23|NULL|   300|         23|           300|         11|
|  NULL|  36|NULL|  NULL|         36|          1270|         11|
+------+----+----+------+-----------+--------------+-----------+



In [56]:
spark.stop()